In [164]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math
import os

In [167]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

In [168]:
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, nhead, num_layers):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(input_dim)
        encoder_layers = nn.TransformerEncoderLayer(input_dim, nhead, dim_feedforward=512)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.encoder = nn.Linear(input_dim, input_dim)
        self.decoder = nn.Linear(input_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, src):
        # print(f"Initial shape: {src.shape}")
        # 预期的输入形状为 (batch_size, sequence_length, input_dim)
        batch_size, sequence_length, input_dim = src.size()

        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        output = self.decoder(output)
        # output = torch.relu(self.decoder(output))
        # 调整回 (batch_size, 240, 200, input_dim, 2, 60)
        output = output.permute(1, 0, 2).contiguous()
        output = output.view(batch_size, 200, input_dim, 2, 60)
        output = self.sigmoid(output)
        return output
    
    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [169]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term[:len(div_term)//2])  # 修改这一行，确保 div_term 维度正确
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return x

In [170]:
def int_to_array(value):
    mapping = {
        0: [0, 0, 0, 0, 0],
        1: [1, 0, 0, 0, 0],
        2: [1, 1, 0, 0, 0],
        3: [1, 1, 1, 0, 0],
        4: [1, 1, 1, 1, 0],
        5: [1, 1, 1, 1, 1]
    }
    return mapping.get(value, [0, 0, 0, 0, 0])

In [172]:
in_seq_length = 1000
df = pd.read_csv('in_seq.csv')
in_seqs = []
for idx, row in df.iterrows():
    in_seq = np.array([int_to_array(int(row[i])) for i in range(1, 1 + in_seq_length)])  # 控制每个in_seq的长度
    in_seqs.append(in_seq)
in_seqs = np.array(in_seqs)
in_seqs = np.transpose(in_seqs, (1, 0, 2))

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10752\678390873.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  in_seq = np.array([int_to_array(int(row[i])) for i in range(1, 1 + in_seq_length)])  # 控制每个in_seq的长度


In [173]:
print(in_seqs.shape)

(1000, 200, 5)


In [174]:
def load_and_pad_npy(file_path, target_shape):
    if os.path.exists(file_path):
        loaded_data = np.load(file_path)
        padded_data = np.zeros(target_shape)
        padded_data[:loaded_data.shape[0], :, :] = loaded_data
        return padded_data
    else:
        return np.zeros(target_shape)

In [175]:
out_seq_length = 1000
out_seqs = []
for i in range(out_seq_length):
    file_path = f"Npy_Output/result_{i+1:03}.npy"
    target_shape = (600, 2, 60)
    data = load_and_pad_npy(file_path, target_shape)
    order = 0
    out_seq = []
    # 根据 in_seqs中的值决定加载的数据
    for j in range(200):
        tmp_seq = []
        for k in range(5):
            tmp = in_seqs[i, j, k]
            if tmp == 1:
                # tmp_seq.append(torch.tensor(data[order]))
                tmp_seq.append(data[order])
                order = order + 1
            else:
                # tmp_seq.append(torch.tensor(np.zeros((2, 60))))
                tmp_seq.append(np.zeros((2, 60)))
        out_seq.append(tmp_seq)
    out_seqs.append(out_seq)
# out_seqs = np.array(out_seqs, dtype=object)
out_seqs = np.array(out_seqs)
print(out_seqs.shape)

(1000, 200, 5, 2, 60)


In [176]:
# 划分数据集为训练集、验证集和测试集
X_train, X_val, Y_train, Y_val = train_test_split(in_seqs, out_seqs, test_size=0.2, random_state=42)
# X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2

# 转换为 torch 张量
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
Y_val = torch.tensor(Y_val, dtype=torch.float32)
# X_test = torch.tensor(X_test, dtype=torch.float32)
# Y_test = torch.tensor(Y_test, dtype=torch.float32)

print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
# print(f"Test set size: {len(X_test)}")

Training set size: 800
Validation set size: 200


In [228]:
input_dim = 5
output_dim = 5 * 2 * 60
nhead = 5
num_layers = 2
lr = 0.001
epochs = 200

model = TransformerModel(input_dim, output_dim, nhead, num_layers)

C:\ProgramData\anaconda3\envs\ECE6258\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [229]:
# # 定义损失函数和优化器
# criterion = nn.MSELoss()
# 
# def custom_loss(output, target):
#     mse_loss = torch.mean((output - target)**2)  # 计算均方误差
#     complement_product_sum = torch.mean(target * (1 - target))  # 计算目标张量中每个元素与其补数的乘积的总和
#     weighted_loss = mse_loss + complement_product_sum
#     return weighted_loss

In [231]:
class CustomLoss(nn.Module):
    def __init__(self, zero_weight=0.1, non_zero_weight=50):
        super(CustomLoss, self).__init__()
        self.zero_weight = zero_weight
        self.non_zero_weight = non_zero_weight
        self.bce_criterion = nn.BCELoss(reduction='none')
        self.mse_criterion = nn.MSELoss(reduction='mean')

    def forward(self, outputs, targets):
        bce_loss = self.bce_criterion(outputs, targets)
        weights = torch.where(targets == 0, self.zero_weight, self.non_zero_weight)
        weighted_bce_loss = bce_loss * weights
        bce_loss_mean = weighted_bce_loss.sum()

        mse_loss = self.mse_criterion(outputs, targets)

        # Calculate sum of (target * (1 - target)) and its mean
        sum_target_times_one_minus_target = 20 * torch.sum(outputs * (1 - outputs))

        # Combine the losses
        final_loss = bce_loss_mean + mse_loss + sum_target_times_one_minus_target

        return final_loss

In [232]:
zero_weight = 0.1 # 负样本的权重
non_zero_weight = 50 # 正样本的权重
criterion = CustomLoss(zero_weight, non_zero_weight)

In [233]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [234]:
# 初始化损失列表
train_losses = []
val_losses = []

# 训练模型
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, Y_train)
    # loss = custom_loss(output, Y_train)  # 如果使用自定义损失函数
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # 在验证集上评估模型
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, Y_val)
        # val_loss = custom_loss(val_output, Y_val)  # 如果使用自定义损失函数
        val_losses.append(val_loss.item())

    if epoch % 1 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

Epoch [1/200], Train Loss: 459257024.0000, Val Loss: 114558128.0000
Epoch [2/200], Train Loss: 458289024.0000, Val Loss: 114300584.0000
Epoch [3/200], Train Loss: 457315200.0000, Val Loss: 114045168.0000
Epoch [4/200], Train Loss: 456350976.0000, Val Loss: 113792800.0000
Epoch [5/200], Train Loss: 455394528.0000, Val Loss: 113545240.0000
Epoch [6/200], Train Loss: 454441440.0000, Val Loss: 113304920.0000
Epoch [7/200], Train Loss: 453512000.0000, Val Loss: 113075512.0000
Epoch [8/200], Train Loss: 452598592.0000, Val Loss: 112857224.0000
Epoch [9/200], Train Loss: 451720480.0000, Val Loss: 112648984.0000
Epoch [10/200], Train Loss: 450884640.0000, Val Loss: 112447592.0000
Epoch [11/200], Train Loss: 450067168.0000, Val Loss: 112247848.0000
Epoch [12/200], Train Loss: 449275104.0000, Val Loss: 112047240.0000
Epoch [13/200], Train Loss: 448479008.0000, Val Loss: 111844880.0000
Epoch [14/200], Train Loss: 447681760.0000, Val Loss: 111640432.0000
Epoch [15/200], Train Loss: 446879552.0000,

In [226]:
# 初始化损失列表
train_losses = []
val_losses = []

# 训练模型
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, Y_train)
    # loss = custom_loss(output, Y_train)  # 如果使用自定义损失函数
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # 在验证集上评估模型
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, Y_val)
        # val_loss = custom_loss(val_output, Y_val)  # 如果使用自定义损失函数
        val_losses.append(val_loss.item())

    if epoch % 1 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

Epoch [1/1000], Train Loss: 13454432.0000, Val Loss: 3506170.0000
Epoch [2/1000], Train Loss: 13458065.0000, Val Loss: 3504981.2500
Epoch [3/1000], Train Loss: 13454423.0000, Val Loss: 3505377.7500
Epoch [4/1000], Train Loss: 13455712.0000, Val Loss: 3505610.7500
Epoch [5/1000], Train Loss: 13456046.0000, Val Loss: 3505635.2500
Epoch [6/1000], Train Loss: 13454964.0000, Val Loss: 3505617.2500
Epoch [7/1000], Train Loss: 13454376.0000, Val Loss: 3505662.5000
Epoch [8/1000], Train Loss: 13454131.0000, Val Loss: 3505583.2500
Epoch [9/1000], Train Loss: 13454513.0000, Val Loss: 3505454.7500
Epoch [10/1000], Train Loss: 13454519.0000, Val Loss: 3505405.2500
Epoch [11/1000], Train Loss: 13453743.0000, Val Loss: 3505483.2500
Epoch [12/1000], Train Loss: 13454217.0000, Val Loss: 3505511.2500
Epoch [13/1000], Train Loss: 13454395.0000, Val Loss: 3505374.5000
Epoch [14/1000], Train Loss: 13454309.0000, Val Loss: 3505215.0000
Epoch [15/1000], Train Loss: 13453652.0000, Val Loss: 3505160.2500
Epoc

In [215]:
# 初始化损失列表
train_losses = []
val_losses = []

# 训练模型
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, Y_train)
    # loss = custom_loss(output, Y_train)  # 如果使用自定义损失函数
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # 在验证集上评估模型
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, Y_val)
        # val_loss = custom_loss(val_output, Y_val)  # 如果使用自定义损失函数
        val_losses.append(val_loss.item())

    if epoch % 1 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

Epoch [1/1000], Train Loss: 21892048.0000, Val Loss: 5463589.5000
Epoch [2/1000], Train Loss: 21794032.0000, Val Loss: 5441181.5000
Epoch [3/1000], Train Loss: 21698898.0000, Val Loss: 5418489.5000
Epoch [4/1000], Train Loss: 21611694.0000, Val Loss: 5396117.0000
Epoch [5/1000], Train Loss: 21528316.0000, Val Loss: 5374543.0000
Epoch [6/1000], Train Loss: 21445600.0000, Val Loss: 5354088.0000
Epoch [7/1000], Train Loss: 21368988.0000, Val Loss: 5336194.5000
Epoch [8/1000], Train Loss: 21311250.0000, Val Loss: 5320557.0000
Epoch [9/1000], Train Loss: 21248264.0000, Val Loss: 5306673.0000
Epoch [10/1000], Train Loss: 21198022.0000, Val Loss: 5293772.5000
Epoch [11/1000], Train Loss: 21143108.0000, Val Loss: 5281340.0000
Epoch [12/1000], Train Loss: 21096304.0000, Val Loss: 5269110.5000
Epoch [13/1000], Train Loss: 21048158.0000, Val Loss: 5256948.5000
Epoch [14/1000], Train Loss: 20997406.0000, Val Loss: 5244780.0000
Epoch [15/1000], Train Loss: 20947648.0000, Val Loss: 5232600.5000
Epoc

In [209]:
# 初始化损失列表
train_losses = []
val_losses = []

# 训练模型
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, Y_train)
    # loss = custom_loss(output, Y_train)  # 如果使用自定义损失函数
    loss.backward()
    optimizer.step()
    train_losses.append(loss.item())

    # 在验证集上评估模型
    model.eval()
    with torch.no_grad():
        val_output = model(X_val)
        val_loss = criterion(val_output, Y_val)
        # val_loss = custom_loss(val_output, Y_val)  # 如果使用自定义损失函数
        val_losses.append(val_loss.item())

    if epoch % 1 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

Epoch [1/1000], Train Loss: 21720032.0000, Val Loss: 5433821.0000
Epoch [2/1000], Train Loss: 21720878.0000, Val Loss: 5433821.0000
Epoch [3/1000], Train Loss: 21719954.0000, Val Loss: 5433821.0000
Epoch [4/1000], Train Loss: 21721182.0000, Val Loss: 5433821.0000
Epoch [5/1000], Train Loss: 21721248.0000, Val Loss: 5433821.0000
Epoch [6/1000], Train Loss: 21719896.0000, Val Loss: 5433821.0000
Epoch [7/1000], Train Loss: 21721418.0000, Val Loss: 5433821.0000
Epoch [8/1000], Train Loss: 21720572.0000, Val Loss: 5433821.0000
Epoch [9/1000], Train Loss: 21723592.0000, Val Loss: 5433821.0000
Epoch [10/1000], Train Loss: 21723378.0000, Val Loss: 5433821.0000
Epoch [11/1000], Train Loss: 21720880.0000, Val Loss: 5433821.0000
Epoch [12/1000], Train Loss: 21721544.0000, Val Loss: 5433821.0000
Epoch [13/1000], Train Loss: 21719988.0000, Val Loss: 5433821.0000
Epoch [14/1000], Train Loss: 21721168.0000, Val Loss: 5433821.0000
Epoch [15/1000], Train Loss: 21720612.0000, Val Loss: 5433821.0000
Epoc

KeyboardInterrupt: 

In [235]:
# 绘制损失曲线
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss Curves')
plt.legend()
plt.savefig('Test_Result_non_zero_50/Test_Case_2/loss_curve_non_zero_50.png')
plt.show()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10752\1265932812.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# 将test case结果输出到csv文件中

In [236]:
in_seq_length = 10
df = pd.read_csv('Test_Instance/in_seq.csv')
in_seqs = []
for idx, row in df.iterrows():
    in_seq = np.array([int_to_array(int(row[i])) for i in range(1, 1 + in_seq_length)])  # 控制每个in_seq的长度
    in_seqs.append(in_seq)
in_seqs = np.array(in_seqs)
in_seqs = np.transpose(in_seqs, (1, 0, 2))
print(in_seqs.shape)

(10, 200, 5)


C:\Users\Administrator\AppData\Local\Temp\ipykernel_10752\1567859702.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  in_seq = np.array([int_to_array(int(row[i])) for i in range(1, 1 + in_seq_length)])  # 控制每个in_seq的长度


In [237]:
test_data = torch.tensor(in_seqs, dtype=torch.float32)
with torch.no_grad():
    test_output = model(test_data)
print(test_output.shape)

torch.Size([10, 200, 5, 2, 60])


In [238]:
test_output_np = test_output.numpy().flatten()
for k in range(10):
    df = pd.DataFrame(test_output_np[120000*k:120000*(k+1)])
    filename = f'Test_Result_non_zero_50/Test_Case_2/test_output_{k+1:03}.csv'
    df.to_csv(filename, index=False, header=False)

# 将test case求解结果输出到csv文件

In [239]:
out_seq_length = in_seq_length
out_seqs = []
for i in range(out_seq_length):
    file_path = f"Test_Instance/Npy_Output/result_{i+1:03}.npy"
    target_shape = (600, 2, 60)
    data = load_and_pad_npy(file_path, target_shape)
    order = 0
    out_seq = []
    # 根据 in_seqs中的值决定加载的数据
    for j in range(200):
        tmp_seq = []
        for k in range(5):
            tmp = in_seqs[i, j, k]
            if tmp == 1:
                # tmp_seq.append(torch.tensor(data[order]))
                tmp_seq.append(data[order])
                order = order + 1
            else:
                # tmp_seq.append(torch.tensor(np.zeros((2, 60))))
                tmp_seq.append(np.zeros((2, 60)))
        out_seq.append(tmp_seq)
    out_seqs.append(out_seq)
# out_seqs = np.array(out_seqs, dtype=object)
out_seqs = np.array(out_seqs)
print(out_seqs.shape)

(10, 200, 5, 2, 60)


In [241]:
for k in range(in_seq_length):
    file_path1 = f"Test_Result_non_zero_50/Test_Case_2/test_output_{k+1:03}.csv"
    existing_df = pd.read_csv(file_path1, header=None)
    file_path2 = f"Test_Instance/Npy_Output/result_{k+1:003}.npy"
    target_shape = (600, 2, 60)
    data = load_and_pad_npy(file_path2, target_shape)
    out_seq = np.array(out_seqs[k]).flatten()
    print(out_seq.shape)
    existing_df[1] = out_seq
    existing_df.to_csv(file_path1, header=False, index=False)

(120000,)
(120000,)
(120000,)
(120000,)
(120000,)
(120000,)
(120000,)
(120000,)
(120000,)
(120000,)
